In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.models import load_model
import os
from time import time
import datetime
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
import multiprocessing as mp
import matplotlib.pyplot as plt 

Using TensorFlow backend.


In [3]:
tensorboard = TensorBoard(log_dir="logs/cpu/{}".format(time()))

In [4]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# CPU Trial

In [6]:
num_CPU=mp.cpu_count()

config = tf.ConfigProto(intra_op_parallelism_threads=8,\
        inter_op_parallelism_threads=8, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : 0})
session = tf.Session(config=config)
K.set_session(session)

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [8]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.8925 - acc: 0.3020 - val_loss: 1.6201 - val_acc: 0.4110
Epoch 2/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.5964 - acc: 0.4162 - val_loss: 1.4496 - val_acc: 0.4695
Epoch 3/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.4753 - acc: 0.4664 - val_loss: 1.6155 - val_acc: 0.4455
Epoch 4/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.3966 - acc: 0.4966 - val_loss: 1.2291 - val_acc: 0.5536
Epoch 5/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.3216 - acc: 0.5285 - val_loss: 1.3016 - val_acc: 0.5484
Epoch 6/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.2647 - acc: 0.5497 - val_loss: 1.1035 - val_acc: 0.6092
Epoch 7/100
1563/1562 [==============================] - 83s 53ms/step - loss: 1.2105 - acc: 0.5703 - val_loss: 1.0897 - val_acc: 0.6175
Epoch 

In [9]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('CPU Runtime:', str(end-start))

10000/10000 [==============================] - 4s 401us/step
Test loss: 0.7184320919036865
Test accuracy: 0.762
CPU Runtime: 8308.942848443985


# GPU Trial

In [5]:
# num_CPU=mp.cpu_count()

# config = tf.ConfigProto(intra_op_parallelism_threads=8,\
#         inter_op_parallelism_threads=8, allow_soft_placement=True,\
#         device_count = {'CPU' : num_CPU, 'GPU' : 1})
# session = tf.Session(config=config)
# K.set_session(session)

K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [6]:
model_gpu = Sequential()
model_gpu.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_gpu.add(Activation('relu'))
model_gpu.add(Conv2D(32, (3, 3)))
model_gpu.add(Activation('relu'))
model_gpu.add(MaxPooling2D(pool_size=(2, 2)))
model_gpu.add(Dropout(0.25))

model_gpu.add(Conv2D(64, (3, 3), padding='same'))
model_gpu.add(Activation('relu'))
model_gpu.add(Conv2D(64, (3, 3)))
model_gpu.add(Activation('relu'))
model_gpu.add(MaxPooling2D(pool_size=(2, 2)))
model_gpu.add(Dropout(0.25))

model_gpu.add(Flatten())
model_gpu.add(Dense(512))
model_gpu.add(Activation('relu'))
model_gpu.add(Dropout(0.5))
model_gpu.add(Dense(num_classes))
model_gpu.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_gpu.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [7]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model_gpu.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_gpu.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size,
                        callbacks=[tensorboard])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 14s 9ms/step - loss: 1.8631 - acc: 0.3180 - val_loss: 1.5886 - val_acc: 0.4270
Epoch 2/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.5920 - acc: 0.4181 - val_loss: 1.3756 - val_acc: 0.4981
Epoch 3/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.4742 - acc: 0.4660 - val_loss: 1.2815 - val_acc: 0.5339
Epoch 4/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.3904 - acc: 0.4999 - val_loss: 1.3028 - val_acc: 0.5452
Epoch 5/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.3234 - acc: 0.5248 - val_loss: 1.2210 - val_acc: 0.5573
Epoch 6/100
1563/1562 [==============================] - 12s 8ms/step - loss: 1.2629 - acc: 0.5502 - val_loss: 1.2083 - val_acc: 0.5653
Epoch 7/100
1563/1562 [==============================] - 13s 9ms/step - loss: 1.2105 - acc: 0.5703 - val_loss: 1.0402 - val_acc: 0.6312
Epoch 8/100
1

In [8]:
model_name = 'gpu_keras_cifar10_trained_model.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model_gpu.save(model_path)

# Score trained model.
scores = model_gpu.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('GPU Runtime:', str(end-start))

10000/10000 [==============================] - 1s 143us/step
Test loss: 0.7379762294769288
Test accuracy: 0.7584
GPU Runtime: 1331.109585762024
